In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
cwd="/content/drive/My Drive/Colab Notebooks"
os.chdir(cwd)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

torch.manual_seed(1)
random.seed(1)

In [7]:
#get inputs of any shape, gets outputs of a certain vector size, simple categorical classification 
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size=1):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_size, hidden_size)      
        self.hidden2out=nn.Linear(hidden_size,output_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.hidden = self.init_hidden()
        self.softmax = nn.LogSoftmax(dim=1) #new
    def forward(self, x): #need to iterate over each item in the input tensor
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        outs = self.out(lstm_out)
        return outs
    def init_hidden(self):
        return (torch.zeros(1, self.batch_size, self.hidden_size),
                torch.zeros(1, self.batch_size, self.hidden_size))
        
n_input=4
n_hidden=16
n_output=15
rnn = RNN(n_input, n_hidden, n_output)
loss_func = nn.MSELoss()
LR=0.005
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
rand_tensor = 0.2*torch.rand((3, 4))
rnn.hidden = rnn.init_hidden()
rnn.zero_grad()
for i in range(len(rand_tensor)): #feed the network sequentially with the input tensors
    cur_tensor=rand_tensor[i].view([1,1,n_input])
    output = rnn(cur_tensor)
print("input shape", rand_tensor.shape)
print("output shape", output.shape)
print(output)


input shape torch.Size([3, 4])
output shape torch.Size([1, 1, 15])
tensor([[[ 0.1337, -0.1304,  0.0239,  0.0808, -0.2139,  0.2093, -0.1822,
           0.2311,  0.2087,  0.0130, -0.0808, -0.0930, -0.0859,  0.0850,
           0.2494]]], grad_fn=<AddBackward0>)


In [9]:
#here the size of the output is the same as the size of the input
#the depth of the output depends on the number of possible outcome categories (e.g. different phonemes)
class RNN(nn.Module):

    def __init__(self, input_size, hidden_size, output_depth,batch_size=1):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size=batch_size
        self.lstm = nn.LSTM(input_size, hidden_size)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2out = nn.Linear(hidden_size, output_depth)
        self.hidden = self.init_hidden()

    def forward(self, feature_list): #emeds are the list of features for each word in the sentece
        #sent_size=len(embeds)
        lstm_out, _ = self.lstm( feature_list.view(len( feature_list), 1, -1))
        tag_space = self.hidden2out(lstm_out.view(len( feature_list), -1))
        #print(tag_space.view([1,1,1]))
        tag_scores = torch.sigmoid(tag_space)
        #return tag_scores
        return tag_space
       
    def init_hidden(self):
        return (torch.zeros(1, self.batch_size, self.hidden_size),
                torch.zeros(1, self.batch_size, self.hidden_size))  

n_input=4
n_hidden =16

rnn = RNN(n_input, n_hidden, 5)
line_tensor=torch.rand((10, 4))
output = rnn(line_tensor)

print("input shape", line_tensor.shape)
print("output shape", output.shape)
print(output)

rnn = RNN(n_input, n_hidden, 1)
line_tensor=torch.rand((10, 4))
output = rnn(line_tensor)

print("input shape", line_tensor.shape)
print("output shape", output.shape)
print(output)


input shape torch.Size([10, 4])
output shape torch.Size([10, 5])
tensor([[ 0.0625,  0.1629, -0.1579, -0.1976, -0.2189],
        [ 0.0632,  0.1552, -0.1676, -0.2049, -0.2173],
        [ 0.0469,  0.1697, -0.1351, -0.1956, -0.2334],
        [ 0.0536,  0.1657, -0.1441, -0.1910, -0.2214],
        [ 0.0481,  0.1689, -0.1473, -0.1953, -0.2241],
        [ 0.0466,  0.1699, -0.1543, -0.2007, -0.2232],
        [ 0.0497,  0.1720, -0.1714, -0.1925, -0.2045],
        [ 0.0445,  0.1714, -0.1446, -0.1927, -0.2214],
        [ 0.0505,  0.1681, -0.1604, -0.1955, -0.2151],
        [ 0.0648,  0.1542, -0.1734, -0.2190, -0.2282]],
       grad_fn=<AddmmBackward>)
input shape torch.Size([10, 4])
output shape torch.Size([10, 1])
tensor([[0.3092],
        [0.3284],
        [0.3348],
        [0.3444],
        [0.3720],
        [0.3787],
        [0.3861],
        [0.3820],
        [0.3635],
        [0.3592]], grad_fn=<AddmmBackward>)


In [10]:
#now we want to just do regression to get one value from a tensor

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size=1):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.lstm = nn.LSTM(input_size, hidden_size)      
        #self.lstm.cuda(cuda0)  
        self.hidden2out=nn.Linear(hidden_size,output_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.hidden = self.init_hidden()
        self.softmax = nn.LogSoftmax(dim=1) #new
        
    def forward(self, x):
        #self.hidden
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        outs = self.out(lstm_out)
        #outs = self.softmax(outs)
        return outs#, h_state


    def init_hidden(self):
        return (torch.zeros(1, self.batch_size, self.hidden_size),
                torch.zeros(1, self.batch_size, self.hidden_size))

n_input=4
n_hidden = 16
rnn = RNN(n_input, n_hidden, 1)
rnn.hidden = rnn.init_hidden()
rnn.zero_grad()
line_tensor=torch.rand((10, 4))

for i in range(line_tensor.size()[0]):
  cur_tensor=line_tensor[i]#.view([(1, 1, 3)])
  cur_tensor=cur_tensor.view([1,1,n_input])
  output = rnn(cur_tensor)
print(output)

tensor([[[0.0117]]], grad_fn=<AddBackward0>)
